#  IMPORTING LIBRARIES 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

# LOADING WEB PAGES WITH 'REQUEST'

In [2]:
url = "https://www.w3schools.com/html/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
page = requests.get(url)

txt = page.text
status = page.status_code

In [3]:
#printing the staus of the requests
print(status)

200


So it's a successful request

# Creating a SOUP object with BEAUTIFULSOUP

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

# MAIN HEADINGS OF HOME PAGE 

In [5]:
# creating a list of all common heading tags
heading_tags = ["h1","h2","h3"]
for tags in soup.find_all(heading_tags):
    print(tags.name + ' -> ' + tags.text.strip())

h2 -> Tutorials
h3 -> HTML and CSS
h3 -> Data Analytics
h3 -> XML Tutorials
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> Web Building
h3 -> Data Analytics
h3 -> XML Tutorials
h2 -> References
h3 -> HTML
h3 -> CSS
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> XML
h3 -> Character Sets
h2 -> Exercises and Quizzes
h3 -> Exercises
h3 -> Quizzes
h3 -> Courses
h3 -> Certificates
h2 -> HTML Tutorial
h2 -> HTML Forms
h2 -> HTML Graphics
h2 -> HTML Media
h2 -> HTML APIs
h2 -> HTML Examples
h2 -> HTML References
h1 -> HTML Tutorial
h2 -> Easy Learning with HTML "Try it Yourself"
h3 -> Example
h2 -> HTML Examples
h2 -> HTML Exercises
h2 -> Test Yourself With Exercises
h2 -> Exercise:
h2 -> HTML Quiz Test
h2 -> HTML References
h2 -> Kickstart your career
h2 -> Report Error
h2 -> Thank You For Helping Us!


# SUB HEADINGS OF HTML 

In [6]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left"><span class="left_h2">HTML</span> Tutorial</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Forms</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Graphics</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Media</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> APIs</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Examples</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> References</h2>]

In [7]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

HTML Tutorial
HTML Forms
HTML Graphics
HTML Media
HTML APIs
HTML Examples
HTML References


In [8]:
# contents in html
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

# Lets scrape the data for the column 'HEADINGS', 'SUB-HEADINGS, 'LINKS', 'DESCRIPTION', 'TABLE DATA', 'UNORDERED LISTS', 'ORDERED LISTS', AND 'OTHER HEADINGS'

----

In [9]:
# getting the Headings, Sub-Headings, Links and Contents

# CREATING RELEVANT VARIABLES
tag = ''
page_table = []
course_heading = []
course_subheading = []
course_links = []
sub_page_content_list = []
unordered_lists = []
ordered_lists = []
list_headings = []
examples_per_page = []
excercises = []
base_url = "https://www.w3schools.com/html/"
course_url = "https://www.w3schools.com/html/"

# FINDING ALL THE LINKS PRESENT IN THE LEFT MENU AND ITERATING OVER THE LINKS TO SCRAPE THE DATA
test = soup.find('div', id='leftmenuinnerinner')

for i in test:
    if 'h2' in str(i):
        tag = i.text
    else:
        if (tag!='') and ('a href' in str(i)) and ('div class="tut_overview"' not in str(i)):
            course_heading.append(tag)
            course_subheading.append(i.text)
            extra = i.get('href')
            if extra[0] == '/':
                base_url = "https://www.w3schools.com"
            
            page_url = base_url + extra
            course_links.append(page_url)
            sub_page = requests.get(page_url)
            soup_page = BeautifulSoup(sub_page.content, 'html.parser')
            
            page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
            page_examples = soup_page.find_all('div', class_ = 'w3-code notranslate htmlHigh')
            
            ##################################################################################################
            
            # Examples per page
            pg_example = []
            for example in page_examples:
                nw = []
                for i in list(example):
                    if str(i) == '<br/>':
                        pass
                    else:
                        nw.append(i)
                pg_example.append(nw)
            examples_per_page.append(pg_example)
                
            ##################################################################################################
            
            # Search for tables 
            tables = soup_page.find_all('table')

            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                for row in all_rows:
                    row_list = row.find_all('td')
                    dataRow = []
                    for cells in row_list:
                        dataRow.append(cells.text)
                    data.append(dataRow)

            table_new = []
            row = ''
            for i in range(1, len(data)):
                if len(data[i]) == 0:
                    row = row[:-3]
                    table_new.append(row) 
                    row = ''
                for j in data[i]:
                    row += j + "|||"
                    # using separator for each column
                row = row[:-3]
                # using '&&&' separator for each row
                row += '&&&'
                #print(row)
            row = row[:-3]
            if len(row) !=0:
                if row[0] == '&':
                    row = row[3:]
            table_new.append(row)
            table_string = ''
            for each in table_new:
                table_string += each + '$$$'
            table_new = table_string[:-3]
            page_table.append(table_new)
            
            ##################################################################################################
            
            # Points for Unordered lists
            ul = soup_page.find_all('ul')
            unorder = []
            for i in ul:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                unorder.append(start_string)
            if len(unorder) == 0:
                unorder = None
            else:
                unorder_string = ''
                for i in unorder:
                    unorder_string += i +'|||'
                unorder = unorder_string[:-3]
            unordered_lists.append(unorder)
            
                        
            ##################################################################################################
            #Points for ordered list.
            ol = soup_page.find_all('ol')
            order = []
            for i in ol:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                order.append(start_string)
            
            if len(order) == 0:
                order = None
            else:
                order_string = ''
                for i in order:
                    order_string += i +'|||'
                order = order_string[:-3]
            ordered_lists.append(order)
            
            ##################################################################################################
            
            # Headings and Subheadings and Description
            
            list_headings_per_page = []
            for page_data in page_content:
                for j in str(page_data).split('\n'):
                    list_headings_per_page.append(j)
                page_paragraphs = page_data.find_all('p')
        
            content_per_page = ''
            for i in page_paragraphs:
                content_per_page += i.text + '|||'
            content_per_page = content_per_page[:-3].replace('\n', '')
            sub_page_content_list.append(content_per_page)
            
            prev = ''
            ul_ol_heading = ''
            for i in list_headings_per_page:
                if ('ul' in i) and (('h2' in prev) or ('h3' in prev)):
                    ul_ol_heading += prev[4:-5] + '|||'
                prev = i
            list_headings.append(ul_ol_heading[:-3])
#             time.sleep(5)      

In [10]:
length = len(course_links)
length

77

In [11]:
len(course_heading), len(course_subheading), len(course_links), len(sub_page_content_list), len(list_headings), len(unordered_lists),len(ordered_lists),  len(page_table), len(examples_per_page)

(77, 77, 77, 77, 77, 77, 77, 77, 77)

# Creating & Saving the dataset into csv file

In [12]:
from csv import writer

# writing the file
with open('html_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('html_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,HTML Tutorial,HTML HOME,https://www.w3schools.com/html/default.asp,HTML is the standard markup language for Web p...,NaN,NaN,NaN,NaN,"[['\n<!DOCTYPE html>', '\n<html>', '<head>', '..."
1,HTML Tutorial,HTML Introduction,https://www.w3schools.com/html/html_intro.asp,HTML is the standard markup language for creat...,What is HTML?|||Example Explained,HTML stands for Hyper Text Markup Language@@@H...,NaN,<h1>|||My First Heading|||</h1>&&&<p>|||My fir...,"[['\n <!DOCTYPE html>', '\n<html>', '<head>', ..."
2,HTML Tutorial,HTML Editors,https://www.w3schools.com/html/html_editors.asp,A simple text editor is all you need to learn ...,"W3Schools Online Editor - ""Try it Yourself""",NaN,NaN,NaN,"[['\n<!DOCTYPE html>', '<html>', '<body>', '<h..."
3,HTML Tutorial,HTML Basic,https://www.w3schools.com/html/html_basic.asp,In this chapter we will show some basic HTML e...,NaN,NaN,NaN,NaN,"[['\n<!DOCTYPE html>', '<html>', '<body>', '<h..."
4,HTML Tutorial,HTML Elements,https://www.w3schools.com/html/html_elements.asp,"An HTML element is defined by a start tag, som...",NaN,NaN,NaN,<h1>|||My First Heading|||</h1>&&&<p>|||My fir...,"[['\n<!DOCTYPE html>', '\n<html>', '<body>', '..."


In [13]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

In [14]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)

In [15]:
# Making a separate column for those heading which are not associated with the Lists
new_col = [None for i in range(len(course_links))]
dataset['Other Headings'] = pd.Series(new_col)
for i in range(len(dataset['Unordered Lists'])):
    if dataset['Unordered Lists'][i] == '':
        dataset['Other Headings'][i] = dataset['List Headings'][i]
        dataset['List Headings'][i] = ''

In [16]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)

In [17]:
# viewing the top 10 rows
dataset.head(10)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples,Other Headings
0,HTML Tutorial,HTML HOME,https://www.w3schools.com/html/default.asp,HTML is the standard markup language for Web p...,,,,,"[['\n<!DOCTYPE html>', '\n<html>', '<head>', '...",
1,HTML Tutorial,HTML Introduction,https://www.w3schools.com/html/html_intro.asp,HTML is the standard markup language for creat...,What is HTML?|||Example Explained,HTML stands for Hyper Text Markup Language@@@H...,,<h1>|||My First Heading|||</h1>&&&<p>|||My fir...,"[['\n <!DOCTYPE html>', '\n<html>', '<head>', ...",
2,HTML Tutorial,HTML Editors,https://www.w3schools.com/html/html_editors.asp,A simple text editor is all you need to learn ...,,,,,"[['\n<!DOCTYPE html>', '<html>', '<body>', '<h...","W3Schools Online Editor - ""Try it Yourself"""
3,HTML Tutorial,HTML Basic,https://www.w3schools.com/html/html_basic.asp,In this chapter we will show some basic HTML e...,,,,,"[['\n<!DOCTYPE html>', '<html>', '<body>', '<h...",
4,HTML Tutorial,HTML Elements,https://www.w3schools.com/html/html_elements.asp,"An HTML element is defined by a start tag, som...",,,,<h1>|||My First Heading|||</h1>&&&<p>|||My fir...,"[['\n<!DOCTYPE html>', '\n<html>', '<body>', '...",
5,HTML Tutorial,HTML Attributes,https://www.w3schools.com/html/html_attributes...,HTML attributes provide additional information...,HTML Attributes|||The width and height Attribu...,All HTML elements can have attributes@@@Attrib...,,,"[['\n<a href=""https://www.w3schools.com"">Visit...",
6,HTML Tutorial,HTML Headings,https://www.w3schools.com/html/html_headings.asp,HTML headings are titles or subtitles that you...,,,,<html>|||Defines the root of an HTML document&...,"[['\n<h1>Heading 1</h1>', '\n<h2>Heading 2</h2...",Bigger Headings
7,HTML Tutorial,HTML Paragraphs,https://www.w3schools.com/html/html_paragraphs...,"A paragraph always starts on a new line, and i...",,,,<p>|||Defines a paragraph&&&<hr>|||Defines a t...,"[['\n<p>This is a paragraph.</p>', '\n<p>This ...",
8,HTML Tutorial,HTML Styles,https://www.w3schools.com/html/html_styles.asp,The HTML style attribute is used to add styles...,Chapter Summary,Use the style attribute for styling HTML eleme...,,,"[['\n <', <em>tagname</em>, '\nstyle=""', <em>p...",
9,HTML Tutorial,HTML Formatting,https://www.w3schools.com/html/html_formatting...,HTML contains several elements for defining te...,,<b> - Bold text@@@<strong> - Important text@@@...,,<b>|||Defines bold text&&&<em>|||Defines empha...,"[['\n <b>This text is bold</b>'], ['\n ...",
